# Set-UID Programs and Environment Variables

## Need for Privileged Programs

- many system programs and configuration files are root/admin owned with restricted access to regular users
- some of these programs need to be executed by regular users regular usage of the system

## Password Dilemma

- let's see the permission of `/etc/shadow` file
- how do regular users change their password?

In [1]:
! ls -al /etc/shadow

-rw-r----- 1 root shadow 1505 Jan 26 15:24 /etc/shadow


In [2]:
! getfacl /etc/shadow

getfacl: Removing leading '/' from absolute path names
# file: etc/shadow
# owner: root
# group: shadow
user::rw-
group::r--
other::---



## Two-Tier Approach

- implementing fine-grained access control in OS make the over complicated
    - requires major redesign/rewrite of all major OS kernels
- OS relies on extension to enforce fine grained access control
- privileged programs are such extensions

![image.png](./media/setuid-approach.png)

- there are two types of privileged programs:

### Daemons
- computer program/process that runs in the background
- typically needs to run as root or other privileged users


### Set-UID programs
- widely used in \*NIX systems
- programs are marked with a speical bit

## Setuid Programs

- setuid programs are a device to allow users to acquire new priviledges for a limited amunt of time
- they provide a meas for overriding the protection schemes designed in \*NIX systems
- `setuid` programs are written in a way that it provides a granual access to individual line of a protected file/resource
- powersuit analogy - suit that gives power to whoever uses it
- which of the following group is the power suit?
    - superman/spiderman suit?
    - ironman/antman suit?
    
- allow users to run a program with the program owner's privilege
- the privilege is temporary while the program is being executed

### Case study - passwd program
- **passwd** program is used to change user's password
- let's see the access control list of **passwd** program
- note `s` in ACL and the owner:group

In [4]:
! ls -l /usr/bin/passwd

-rwsr-xr-x 1 root root 72344 Jan  5 19:57 /usr/bin/passwd


In [5]:
! getfacl /usr/bin/passwd

getfacl: Removing leading '/' from absolute path names
# file: usr/bin/passwd
# owner: root
# group: root
# flags: s--
user::rwx
group::r-x
other::r-x



## Set-UID Concept

- every process has two User IDs:

#### Real UID (RUID)
- identifies the real owner of the process

#### Effective UID (EUID)
- identifies privilege of a process
- access control is based on **EUID**

- when a normal program is executed, **RUID = EUID**
    - both IDs equal to the ID of the user who runs the program
- when a **set-uid** is executed, **RUID != EUID**
    - RUID still equals to user's ID who runs the program
    - EUID howover is equal to the program owner's ID
    - if the program is owned by root, the program runs with the root privilege
    
### meowcat - copy cat
- create a program that behaves like the `cat` program
- the demo program is provided in `demos/setuid` folder
- let's change the working diectory to `demos/setuid`
- use jupyter notebook's magic symbol to do that: `%`

In [35]:
%pwd

'/home/kali/SoftwareSecurity/demos/setuid'

In [7]:
%cd demos/setuid

/home/kali/SoftwareSecurity/demos/setuid


In [9]:
# if you get a warning when using magic symbol...
# UserWarning: using dhist requires you to install the `pickleshare` library.
! pip install pickleshare

In [10]:
! ls -al

total 60
drwxr-xr-x  2 kali kali  4096 Jan 31 13:23 .
drwxr-xr-x 17 kali kali  4096 Jan 24 15:20 ..
-rw-r--r--  1 kali kali   443 Jan 31 13:23 Makefile
-rw-r--r--  1 kali kali    15 Jan 24 15:20 data.txt
-rwxr-xr-x  1 kali kali 37160 Jan 24 15:20 meowcat
-rw-r--r--  1 kali kali   911 Jan 24 15:20 meowcat.cpp


In [11]:
! cat meowcat.cpp

// Program that mimics bash cat program
#include <iostream>
#include <cstdio>
#include <fstream>
#include <string>

using namespace std;

int main(int argc, char* argv[]) {
    if (argc <= 1) {
        printf("Usage: program <filepath>\n");
        return 1;
    }
    ifstream fin;
    fin.open(argv[1], fin.in | fin.binary | fin.ate);

    if (!fin.is_open())
        cout << "Failed to open " << argv[1] << '\n';
    else {
        // findout the size of the the file; get position in input sequence
        size_t size = fin.tellg();
        // Set position in input sequence
        fin.seekg(0, fin.beg);
        // allocate memory to store file contents
        char * buffer = new char[size];
        
        if (fin.read(buffer, size))
        {
            cout << buffer << endl;
            // parse buffer in memory...
        }
        delete[] buffer;
        fin.close();
    }
    return 0;
}


In [36]:
! cat Makefile

COMPILER = g++

COMPILER_FLAGS = -g -Wall -std=c++17

# the build target executable program name of your choice
PROGRAM_NAME = meowcat

# list all .cpp files separated by space; files must exist!
CPP_FILES = meowcat.cpp

.PHONY: build
build:
	$(COMPILER) $(COMPILER_FLAGS) $(CPP_FILES) -o $(PROGRAM_NAME)
	# # make clean ruleudo chown root:root $(PROGRAM_NAME)
	# sudo chmod +s $(PROGRAM_NAME)

.PHONY: clean
clean:
	rm -f $(PROGRAM_NAME) *.o


In [37]:
! make

g++ -g -Wall -std=c++17 meowcat.cpp -o meowcat
# # make clean ruleudo chown root:root meowcat
# sudo chmod +s meowcat


In [14]:
! ./meowcat

Usage: program <filepath>


In [40]:
! ls -al

total 60
drwxr-xr-x  2 kali kali  4096 Feb  2 15:09 .
drwxr-xr-x 17 kali kali  4096 Jan 24 15:20 ..
-rw-r--r--  1 kali kali   443 Jan 31 13:23 Makefile
-rw-r--r--  1 kali kali    15 Jan 31 15:46 data.txt
-rwxr-xr-x  1 kali kali 37152 Feb  2 15:09 meowcat
-rw-r--r--  1 kali kali   911 Jan 24 15:20 meowcat.cpp


In [41]:
! getfacl meowcat

# file: meowcat
# owner: kali
# group: kali
user::rwx
group::r-x
other::r-x



In [42]:
! echo "hello there..." > data.txt

In [43]:
! cat data.txt

hello there...


In [44]:
! ./meowcat data.txt

hello there...



In [45]:
! ./meowcat /etc/shadow

Failed to open /etc/shadow


In [19]:
! ls -al 

total 60
drwxr-xr-x  2 kali kali  4096 Jan 31 15:45 .
drwxr-xr-x 17 kali kali  4096 Jan 24 15:20 ..
-rw-r--r--  1 kali kali   443 Jan 31 13:23 Makefile
-rw-r--r--  1 kali kali    15 Jan 31 15:46 data.txt
-rwxr-xr-x  1 kali kali 37152 Jan 31 15:45 meowcat
-rw-r--r--  1 kali kali   911 Jan 24 15:20 meowcat.cpp


In [47]:
! getfacl $(which sudo)

getfacl: Removing leading '/' from absolute path names
# file: usr/bin/sudo
# owner: root
# group: root
# flags: s--
user::rwx
group::r-x
other::r-x



In [48]:
# let's set the ownership of the program to root to setuid
! echo kali | sudo -S chown root meowcat

[sudo] password for kali: 

In [49]:
! ls -al meowcat

-rwxr-xr-x 1 root kali 37152 Feb  2 15:09 meowcat


In [50]:
! getfacl meowcat

# file: meowcat
# owner: root
# group: kali
user::rwx
group::r-x
other::r-x



In [51]:
# let's setuid chmod 4755
! echo kali | sudo -S chmod u+s meowcat

[sudo] password for kali: 

In [52]:
! ls -al meowcat

-rwsr-xr-x 1 root kali 37152 Feb  2 15:09 meowcat


In [54]:
! cat data.txt

hello there...


In [56]:
! ./meowcata data.txt

zsh:1: no such file or directory: ./meowcata


In [57]:
# if the owner isn't root or privileged user, the program is setuid but NOT privilege
! echo kali | sudo -S chown user meowcat

[sudo] password for kali: 

In [58]:
! ls -al meowcat

-rwxr-xr-x 1 user kali 37152 Feb  2 15:09 meowcat


In [59]:
! echo kali | sudo -S chmod 4777 meowcat

[sudo] password for kali: 

In [60]:
! ls -al meowcat

-rwsrwxrwx 1 user kali 37152 Feb  2 15:09 meowcat


In [61]:
! ./meowcat /etc/shadow

Failed to open /etc/shadow


## What programs need setuid?

- programs that require to access sensitive, system-wide data, files, configs, etc.
- is it safe to setuid every programs?
    - /bin/cat, /bin/sh, nano, vi, etc.


## Environment Variables

- environment variables are integral part of all OS
- these are dynamic values that can have significant influence over the behavior of programs and processes
- these variables are outside the program that store configuration or settings and their values can be changed without modifying the programs' actual code 
- one of the most important `env` variable is `PATH`
    - PATH stores the locations to search for various binaries, libraries, and executable programs
- programs and scripts can access environment variables to adapt their behavior based on the configuration and requirements of the system they're being executed
- OSes provide various ways to set and access environment variables
- system APIs are available to programming languages to access and set environment variables in the programs
- in Linux, env variables play important roles when using Terminals to run programs
- `env` or `printenv` commands list all the environment variables available to the current user
- variable names are case sensitive
- normally ALL_CAPS are used for env variable names

In [ ]:
#! /usr/bin/env python

print ('hello world')

In [64]:
! /usr/bin/env python

Python 3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:43:09) [GCC 12.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 0, in <module>
KeyboardInterrupt
>>> 

### Setting and Accessing Environment Variables

- env variables can be set in many places depending on the needs

1. System wide
    - /etc/environment
    - /etc/profile
2. User specific
    - ~/.profile
    - ~/.bashrc
    - ~/.zshrc

In [65]:
! cat /etc/environment

# START KALI-DEFAULTS CONFIG
# Everything from here and until STOP KALI-DEFAULTS CONFIG
# was installed by the kali-defaults package, and it will
# be removed if ever the kali-defaults package is removed.
# If you want to disable a line, please do NOT remove it,
# as it would be added back when kali-defaults is upgraded.
# Instead, comment the line out, and your change will be
# preserved across upgrades.
PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games
COMMAND_NOT_FOUND_INSTALL_PROMPT=1
POWERSHELL_UPDATECHECK=Off
POWERSHELL_TELEMETRY_OPTOUT=1
DOTNET_CLI_TELEMETRY_OPTOUT=1
# STOP KALI-DEFAULTS CONFIG


In [67]:
! echo $HOME

/home/kali


In [68]:
! cat $HOME/.zshrc

# ~/.zshrc file for zsh interactive shells.
# see /usr/share/doc/zsh/examples/zshrc for examples

setopt autocd              # change directory just by typing its name
#setopt correct            # auto correct mistakes
setopt interactivecomments # allow comments in interactive mode
setopt magicequalsubst     # enable filename expansion for arguments of the form ‘anything=expression’
setopt nonomatch           # hide error message if there is no match for the pattern
setopt notify              # report the status of background jobs immediately
setopt numericglobsort     # sort filenames numerically when it makes sense
setopt promptsubst         # enable command substitution in prompt

WORDCHARS=${WORDCHARS//\/} # Don't consider certain characters part of the word

# hide EOL sign ('%')
PROMPT_EOL_MARK=""

# configure key keybindings
bindkey -e                                        # emacs key bindings
bindkey ' ' magic-space                           # do history expansion on space
bin

In [69]:
! echo $PATH

/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games


### Types of Env Variables

#### shell variable
- like local variables in programs; available to the current shell until it's terminated
- can't have trailing and leading space around the '=' sign unlike in many programming languages
- space is a special character -- word/command separater -- in bash scripting

```bash
VAR_NAME="VALUE..."
```

#### environment or system variables
- like global variables; they can be available system-wide across the Terminals
- temporary env variable can be exported from the Terminal and availble only to that Terminal
- permenant env variables are exported from init script such as `~/.zshrc` everytime a Terminal is launched

```bash
export VAR[="VALUE..."]
set VAR1="some other values"
```

- can't create shell and environemnt variables from Jupyter Notebooks
- use Terminal instead...
- add `$` before the variable name when reading the value

In [70]:
! VAR=test

In [71]:
! echo $VAR

```bash
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/setuid]
└─$ VAR="Some data"
                                                                                                     
(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/setuid]
└─$ echo $VAR            
Some data

(base) ┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/setuid]
└─$ cd             
                                                                                                     
(base) ┌──(kali㉿kali)-[~]
└─$ echo $VAR
Some data

(base) ┌──(kali㉿kali)-[~]
└─$ export VAR
                                                                                                     
(base) ┌──(kali㉿kali)-[~]
└─$ env | grep VAR
VAR=Some data
```

### Update existing env variable

- sometime we may need to temporarily and dynamically add a new path to the PATH variable

```bash
(base) ┌──(kali㉿kali)-[~]
└─$ PATH=$PATH:$HOME/bin
                                                                                                     
(base) ┌──(kali㉿kali)-[~]
└─$ export PATH 

# one liner
(base) ┌──(kali㉿kali)-[~]
└─$ export PATH=$PATH:$HOME/bin

(base) ┌──(kali㉿kali)-[~]
└─$ env | grep ^PATH=
PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games:/home/kali//home/kali/bin
```

### Delete environment variables

- can use `unset` command or assign empty value to temporarily delete them
- can update the corresponding setting files to permanently delete them

```
unset NAME
NAME=""
```

# How to access environment variables from C/C++ programs

- you can pass `*char []` to the main as the third argument
- you can use global varaiable `environ`
- you can use system API such as `getenv(var_name)`, `putenv()`, `setenv(), unsetenv()`, etc.
- let's see how they works
- example codes provided in `demos/envvars` folder

In [1]:
%pwd

'/home/kali/projects/SoftwareSecurity'

In [2]:
%cd demos/envvars

/home/kali/projects/SoftwareSecurity/demos/envvars


In [3]:
! cat pass_env.cpp

#include <cstdio>

// envp[] can only be used within the main program 
// needs to be passsed to other functions as needed
int main(int argc, char* argv[], char* envp[]) {
    printf("Total argument count = %d\n", argc);
    for(int i=0; i < argc; i++){
        printf("arg[%d] = %s\n", i, argv[i]);
    }
    printf("Now printing environment variables accessing envp array:\n");
    int i = 0;
    while (envp[i] != NULL) {
        printf("%s\n", envp[i++]);
    }
    return 0;
}

In [4]:
! g++ -o pass_env pass_env.cpp

In [5]:
! ./pass_env hi there world!

Total argument count = 4
arg[0] = ./pass_env
arg[1] = hi
arg[2] = there
arg[3] = world!
Now printing environment variables accessing envp array:
SHELL=/usr/bin/zsh
SESSION_MANAGER=local/kali:@/tmp/.ICE-unix/1083,unix/kali:/tmp/.ICE-unix/1083
WINDOWID=0
QT_ACCESSIBILITY=1
COLORTERM=truecolor
GCC_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib
XDG_CONFIG_DIRS=/etc/xdg
XDG_SESSION_PATH=/org/freedesktop/DisplayManager/Session0
XDG_MENU_PREFIX=xfce-
CONDA_EXE=/home/kali/miniconda3/bin/conda
_CE_M=
build_alias=x86_64-conda-linux-gnu
CMAKE_ARGS=-DCMAKE_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ar -DCMAKE_CXX_COMPILER_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ar -DCMAKE_C_COMPILER_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ar -DCMAKE_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ranlib -DCMAKE_CXX_COMPILER_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib -DCMAKE_C_COMPILER_RANLIB=/home/kali/miniconda3/bin/x86_6

In [6]:
! cat global_env.cpp

#include <cstdio>

extern char** environ;

int main(int argc, char* argv[]) {
    int i = 0;
    while (environ[i] != NULL) {
        printf("%s\n", environ[i++]);
    }
    return 0;
}


In [8]:
! g++ -o global_env global_env.cpp

In [9]:
! ./global_env

SHELL=/usr/bin/zsh
SESSION_MANAGER=local/kali:@/tmp/.ICE-unix/1083,unix/kali:/tmp/.ICE-unix/1083
WINDOWID=0
QT_ACCESSIBILITY=1
COLORTERM=truecolor
GCC_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib
XDG_CONFIG_DIRS=/etc/xdg
XDG_SESSION_PATH=/org/freedesktop/DisplayManager/Session0
XDG_MENU_PREFIX=xfce-
CONDA_EXE=/home/kali/miniconda3/bin/conda
_CE_M=
build_alias=x86_64-conda-linux-gnu
CMAKE_ARGS=-DCMAKE_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ar -DCMAKE_CXX_COMPILER_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ar -DCMAKE_C_COMPILER_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ar -DCMAKE_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ranlib -DCMAKE_CXX_COMPILER_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib -DCMAKE_C_COMPILER_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib -DCMAKE_LINKER=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ld -DCMAKE_STRIP=/home/kali/miniconda3/bin/x86_64-co

In [10]:
! cat get_set_env.cpp

#include <cstdio>
#include <stdlib.h> // getenv, putenv, setenv, unsetenv, etc.
// see: http://www.manpagez.com/man/3/getenv/

int main(int argc, char* argv[]) {
  if (argc == 2) {
    // read value of env variable
    //getenv(char * name) retruns NUL-terminated string; if name doesn't exists, returns NULL
    printf("%s=%s\n", argv[1], getenv(argv[1]));
  }
  else if (argc == 3) { 
    // set env variable, 0 doesn't overwrite, 1 overwrites
    int result = setenv(argv[1], argv[2], 0); // returns 0 on successful, -1 otherwise
    if (result == 0) {
      printf("Successfully set env variable!\n");
      printf("%s=%s\n", argv[1], getenv(argv[1]));
      // you can't use setenv to export variables from current process to the calling process (shell)
    }
    else {
      printf("Couldn't set env variable!");
    }
  }
  else {
    printf("%s\n", "Usage READ: program ENV_VAR_NAME");
    printf("%s\n", "Usage SET: program ENV_VAR_NAME VALUE");
  }
  return 0;
}


In [11]:
! g++ -o get_set_env get_set_env.cpp

In [13]:
! ./get_set_env PATH

PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games


In [14]:
! ./get_set_env TEST "HELLO THERE"

Successfully set env variable!
TEST=HELLO THERE


In [15]:
# rerunning the program doesn't find the newly set environement variable
# the program is rerun everytime...
! ./get_set_env TEST

TEST=(null)


## How a process gets its environment variables

- one of the two ways
- if a new process is created using `fork()`, the parent process copies and passes its environment variables to the child process
- if a process runs a new process in itself (using `execve()`), the processe's environment variable is overwritten by the new env vars provided to the new process
    - current environment variables are essentially lost/overwritten
- `execve()` system API has 3 parameters
    - `filename, argv, envp`
- https://man7.org/linux/man-pages/man2/execve.2.html

```cpp
execv(const char* filename, char *const argv[], char *const envp[]);
```
- let's see how `execve` can pass the env variables to processes

In [23]:
! cat execve_passenv.cpp

#include <cstdio>
#include <unistd.h>

extern char** environ;

int main(int argc, char* argv[], char* envp[]) {
    int i = 0;
    char* arg[2];
    char* newenv[3];
    if (argc < 2) {
        printf("Missing required argument 1, 2, 3");
        printf("Usage: ./execve_passenv [1/2/3]");
        return 1;
    }

    // construct the argument array
    arg[0] = "/usr/bin/env"; arg[1] = NULL;

    // construct the environment variable array
    newenv[0] = "AAA=aaa";
    newenv[1] = "BBB=123";
    newenv[2] = NULL;
    switch(argv[1][0]) {
        case '1': // passing no env variable
            execve(arg[0], arg, NULL);
            break;
        case '2': // passing new env variables
            execve(arg[0], arg, newenv); 
            break;
        case '3': // passing current env variables
            execve(arg[0], arg, environ);
            break;
        default: // passing no env variable
            execve(arg[0], arg, NULL);
    }

}


In [24]:
! g++ -o execve_passenv execve_passenv.cpp

execve_passenv.cpp: In function ‘int main(int, char**, char**)’:
execve_passenv.cpp:17:14: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   17 |     arg[0] = "/usr/bin/env"; arg[1] = NULL;
      |              ^~~~~~~~~~~~~~
execve_passenv.cpp:20:17: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   20 |     newenv[0] = "AAA=aaa";
      |                 ^~~~~~~~~
execve_passenv.cpp:21:17: warning: ISO C++ forbids converting a string constant to ‘char*’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wwrite-strings-Wwrite-strings]8;;]
   21 |     newenv[1] = "BBB=123";
      |                 ^~~~~~~~~


In [25]:
# no envvariables are printed for default and option 1
! ./execve_passenv

Missing required argument 1, 2, 3Usage: ./execve_passenv [1/2/3]

In [26]:
! ./execve_passenv 1

In [27]:
! ./execve_passenv 2

AAA=aaa
BBB=123


In [28]:
! ./execve_passenv 3

SHELL=/usr/bin/zsh
SESSION_MANAGER=local/kali:@/tmp/.ICE-unix/1083,unix/kali:/tmp/.ICE-unix/1083
WINDOWID=0
QT_ACCESSIBILITY=1
COLORTERM=truecolor
GCC_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib
XDG_CONFIG_DIRS=/etc/xdg
XDG_SESSION_PATH=/org/freedesktop/DisplayManager/Session0
XDG_MENU_PREFIX=xfce-
CONDA_EXE=/home/kali/miniconda3/bin/conda
_CE_M=
build_alias=x86_64-conda-linux-gnu
CMAKE_ARGS=-DCMAKE_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ar -DCMAKE_CXX_COMPILER_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ar -DCMAKE_C_COMPILER_AR=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ar -DCMAKE_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ranlib -DCMAKE_CXX_COMPILER_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib -DCMAKE_C_COMPILER_RANLIB=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-gcc-ranlib -DCMAKE_LINKER=/home/kali/miniconda3/bin/x86_64-conda-linux-gnu-ld -DCMAKE_STRIP=/home/kali/miniconda3/bin/x86_64-co

## Memory Locations for program arguments and environment variables

- let's see the stack frame of main function

```

## Futher Readings and References
- How to Write Setuid Programs - https://nob.cs.ucdavis.edu/~bishop/secprog/1987-sproglogin.pdf
- Setuid BSDI Man page - https://seedsecuritylabs.org/Labs_20.04/Software/Environment_Variable_and_SetUID/files/setuid.pdf
- Setuid Demystified - https://web.ecs.syr.edu/~wedu/minix/projects/setuid_paper.pdf